In [1]:
#import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#import data set
df = pd.read_csv('nba_data.csv')

In [3]:
#drop and rename columns
df.drop('RANK', axis = 1, inplace = True)
df.rename(columns= {
    'FULL NAME' : 'NAME', 
    'MIN%Minutes PercentagePercentage of team minutes used by a player while he was on the floor' : 'MIN%',
    'USG%Usage RateUsage rate, a.k.a., usage percentage is an estimate of the percentage of team plays used by a player while he was on the floor' : 'USG%',
    'TO%Turnover RateA metric that estimates the number of turnovers a player commits per 100 possessions' : 'TO%',
    'eFG%Effective Shooting PercentageWith eFG%, three-point shots made are worth 50% more than two-point shots made. eFG% Formula=(FGM+ (0.5 x 3PM))/FGA' : 'eFG%',
    'TS%True Shooting PercentageTrue shooting percentage is a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws.' : 'TS%',
    'PPGPointsPoints per game.' : 'PPG',
    'RPGReboundsRebounds per game.' : 'RPG',
    'TRB%Total Rebound PercentageTotal rebound percentage is estimated percentage of available rebounds grabbed by the player while the player is on the court.' : 'TRB%',
    'APGAssistsAssists per game.' : 'APG',
    'AST%Assist PercentageAssist percentage is an estimated percentage of teammate field goals a player assisted while the player is on the court' : 'AST%',
    'SPGStealsSteals per game.' : 'SPG',
    'BPGBlocksBlocks per game.' : 'BPG',
    'TOPGTurnoversTurnovers per game.' : 'TOPG',
    'VIVersatility IndexVersatility index is a metric that measures a player’s ability to produce in points, assists, and rebounds. The average player will score around a five on the index, while top players score above 10' : 'VI',
    'ORTGOffensive RatingIndividual offensive rating is the number of points produced by a player per 100 total individual possessions.' : 'ORTG',
    'DRTGDefensive RatingIndividual defensive rating estimates how many points the player allowed per 100 possessions he individually faced while staying on the court.' : 'DRTG'
}, inplace = True)

In [4]:
df_dupes = df[df.duplicated(['NAME'], keep=False)] #make dataframe containing only duplicate players
df_dupes.sort_values(by = ['NAME'], inplace = True) #sort dataframe by name

'''
get name of first row in list
compare next row name with previous row name
if they have the same name. continue to next row
if they have different names, make a weighted average of stats of previous rows
add weighted stats to a list (should be a list of dictionaries
'''

num_of_dupes = 0 #variable keeps track of how many dupes, players can get traded more than once per season
list_of_new_plyr_stats = [] #variable to add new player stat dictionaries too

for i in range(len(df_dupes)):
    if (i == len(df_dupes)-1) or df_dupes.iloc[i]['NAME'] != df_dupes.iloc[i+1]['NAME']:
        new_plyr_stats = {
            #use same name, team, position and age, last team in last is the one the finished season on
            'NAME' : df_dupes.iloc[i]['NAME'],
            'TEAM' : df_dupes.iloc[i]['TEAM'],
            'POS' : df_dupes.iloc[i]['POS'],
            'AGE' : df_dupes.iloc[i]['AGE']
        }
        
        #calculate total games played
        total_games = 0
        for j in range(num_of_dupes, -1, -1):
            total_games = total_games + df_dupes.iloc[i-j]['GP']
        new_plyr_stats['GP'] = total_games
        
        #calculate mean of average of all stats except FTA, 2PA, 3PA
        #create list of columns we want to calculate new value of
        mean_avg_col = list(df_dupes.columns)
        for j in ['NAME', 'TEAM', 'POS', 'AGE', 'GP']:
            mean_avg_col.remove(j)
        
        #iterate through column list and calculate new value
        for j in mean_avg_col:
            new_stat = 0
            
            #weighted average is not taken for these stats, just added together
            if j == 'FTA' or j == '2PA' or j == '3PA':
                for k in range(num_of_dupes, -1, -1):
                    new_stat = new_stat + (df_dupes.iloc[i-k][j])
            
            #calculate weighted average using number of games played on each team
            else:
                for k in range(num_of_dupes, -1, -1):
                    new_stat = new_stat + (df_dupes.iloc[i-k][j])*(df_dupes.iloc[i-k]['GP']/new_plyr_stats['GP'])
                    
                    #round ceratin categories to 1, 2, or 3 decimal places
                    if j == 'SPG' or j == 'BPG' or j == 'TOPG':
                        new_stat = round(new_stat, 2)
                    elif j == 'FT%' or j == '2P%' or j == '3P%' or j == 'eFG%' or j == 'TS%':
                        new_stat = round(new_stat, 3)
                    else:
                        new_stat = round(new_stat, 1)
            
            #add new stat to dictionary
            new_plyr_stats[j] = new_stat
        
        #reset number of dupes for new player
        num_of_dupes = 0
        
        #add new calculated stats to list of duplicated players
        list_of_new_plyr_stats.append(new_plyr_stats)
        
        
    else:
        num_of_dupes = num_of_dupes + 1
        #df_dupe_player = df_dupes[(df_dupes.NAME != df_dupes.iloc[i]['NAME'])]

/Users/christophercorallo/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [5]:
df.drop_duplicates(subset = 'NAME', keep = False, inplace = True) #drop duplicates from original dataframe

#add list of new players with new stats to dataframe
df1 = pd.DataFrame(list_of_new_plyr_stats)
df = pd.concat([df, df1])

df['AvgFanPt'] = (df['PPG'] + df['RPG']*1.2 + df['APG']*1.5 + df['SPG']*3 + df['BPG']*3 - df['TOPG'])
df.reset_index(inplace = True)
df.drop(['index'], axis = 1, inplace = True)
df

,NAME,TEAM,POS,AGE,GP,MPG,MIN%,USG%,TO%,FTA,...,TRB%,APG,AST%,SPG,BPG,TOPG,VI,ORTG,DRTG,AvgFanPt
0,Aaron Holiday,Ind,G,24.63,66,17.8,37.1,19.5,12.3,83,...,4.1,1.9,14.1,0.70,0.20,1.00,5.9,98.6,110.7,13.31
1,Aaron Nesmith,Bos,G-F,21.58,46,14.5,30.3,13.7,10.8,28,...,10.6,0.5,4.6,0.33,0.20,0.50,5.2,112.3,102.6,9.90
2,Abdel Nader,Pho,F,27.64,24,14.8,30.8,19.0,12.6,37,...,9.9,0.8,7.3,0.42,0.38,0.79,6.5,110.4,104.1,12.63
3,Adam Mokoka,Chi,G,22.83,14,4.0,8.4,18.9,20.5,1,...,4.9,0.4,11.9,0.14,0.07,0.36,5.6,72.9,107.5,2.45
4,Al Horford,Okc,C-F,34.95,28,27.9,58.2,21.6,7.3,22,...,12.5,3.4,20.1,0.89,0.93,1.04,9.8,111.2,109.0,31.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,Tyler Cook,Bro,F,23.65,32,13.6,28.5,13.7,16.6,35,...,11.6,0.5,6.7,0.25,0.06,0.69,5.5,113.6,107.8,9.61
536,Victor Oladipo,Mia,G,29.03,33,32.8,68.1,28.5,11.8,138,...,8.0,4.5,22.7,1.40,0.42,2.52,9.3,99.6,109.1,35.37
537,Wendell Carter Jr.,Orl,C-F,22.09,54,25.4,53.0,19.3,12.8,179,...,17.3,2.0,11.3,0.64,0.77,1.45,8.9,113.5,103.0,26.92
538,Wes Iwundu,Nor,F,26.41,41,13.1,27.4,10.8,13.4,31,...,9.0,0.4,4.2,0.36,0.10,0.47,3.9,89.5,106.9,6.55


In [6]:
df2 = df.drop(['TEAM', 'POS', 'AGE', 'GP', 'MPG', 'MIN%', 'USG%', 'TO%', 'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 
         'eFG%', 'TS%', 'TRB%', 'AST%', 'VI', 'ORTG','DRTG'], axis = 1)
df2 = df2.round({'SPG' : 1, 'BPG' : 1, 'TOPG' : 1})
df2['AvgFanPt'] = (df2['PPG'] + df2['RPG'] * 1.2 + df2['APG'] * 1.5 + df2['SPG'] * 3 + df2['BPG'] * 3 - df2['TOPG'])
df2.sort_values(by = ['AvgFanPt'], ascending = False, inplace = True)
df2.head(10)

,NAME,PPG,RPG,APG,SPG,BPG,TOPG,AvgFanPt
343,Nikola Jokic,26.4,10.8,8.3,1.3,0.7,3.1,54.71
386,Russell Westbrook,22.2,11.5,11.7,1.4,0.4,4.8,54.15
153,Giannis Antetokounmpo,28.1,11.0,5.9,1.2,1.2,3.4,53.95
498,James Harden,24.6,7.9,10.8,1.2,0.8,4.0,52.28
290,Luka Doncic,27.7,8.0,8.6,1.0,0.6,4.3,50.70
217,Joel Embiid,28.5,10.5,2.8,1.0,1.4,3.1,49.40
402,Stephen Curry,32.0,5.5,5.8,1.2,0.1,3.4,47.80
285,LeBron James,25.0,7.7,7.8,1.1,0.6,3.7,47.34
246,Karl-Anthony Towns,24.8,10.6,4.5,0.8,1.1,3.2,46.77
115,Domantas Sabonis,20.3,12.0,6.7,1.2,0.5,3.4,46.45
